In [1]:
## Importing important libraries

import netCDF4
from netCDF4 import Dataset

import pandas as pd
import numpy as np

import glob

In [2]:
class netCDF:
    
    def __init__(self):
        self.path = input('Enter the path of netCDF file : ')
        self.lat = float(input('Enter the Latitude of your location : '))
        self.lon = float(input('Enter the Longitude of your location : '))
        
        # Record all the years of the netCDF files into a Python list
        self.all_years = []

        for file in glob.glob(self.path + '/*.nc'):
            year = file.split('.')[1]
            self.all_years.append(year)
        
        # Creating an empty Pandas DataFrame covering the whole range of data 
        self.year_start = min(self.all_years) 
        self.end_year = max(self.all_years)
        self.date_range = pd.date_range(start = str(self.year_start) + '-01-01', 
                                   end = str(self.end_year) + '-12-31', 
                                   freq = 'D')
        
        self.df = pd.DataFrame(0.0, columns = ['Max_Temparature'], index = self.date_range)
            
        
    def extract_maxTemp(self):
        
        for yr in self.all_years:
            
            # Reading-in the data 
            data = Dataset(self.path+'/tmax.'+str(yr)+'.nc', 'r')
    
            # Storing the lat and lon data of the netCDF file into variables 
            lat_data = data.variables['lat'][:]
            lon_data = data.variables['lon'][:]
    
            # Absolute difference of lat and lon -to get the closest value of lat and lon
            abs_diff_lat = np.absolute(lat_data - self.lat)
            abs_diff_lon = np.absolute(lon_data - self.lon)
    
            # Identify the index of the min value for lat and lon
            min_index_lat = abs_diff_lat.argmin()
            min_index_lon = abs_diff_lon.argmin()
    
            # Accessing the Max temparature data
            tMax = data.variables['tmax']
    
            # Creating the date range for each year during each iteration
            start = str(yr) + '-01-01'
            end = str(yr) + '-12-31'
            d_range = pd.date_range(start = start, 
                                    end = end, 
                                    freq = 'D')
    
            for t_index in np.arange(0, len(d_range)):            
                self.df.loc[d_range[t_index]]['Max_Temparature'] = tMax[t_index, min_index_lat, min_index_lon]
        return self.df
    
    def netCDFtoCSV(self):
        f_name = input('Enter the file name : ')
        self.df.to_csv('{}.csv'.format(f_name))

In [3]:
netCDF().extract_maxTemp()

Enter the path : D:\Python\Python Program\Temp Extracting using latitude and logitude\tmax_file
Enter the Latitude of your location : 25.5941
Enter the Longitude of your location : 85.1376


D:\Python\Python3.10.4 Installed\lib\site-packages\pandas\core\internals\base.py:190: UserWarning: Warning: converting a masked element to nan.
  arr[indexer] = value


,Max_Temparature
1979-01-01,22.754578
1979-01-02,24.716000
1979-01-03,24.802210
1979-01-04,24.962790
1979-01-05,24.760494
...,...
2022-12-27,22.885139
2022-12-28,18.098227
2022-12-29,20.740852
2022-12-30,19.786230
